In [ ]:
#import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import pymongo 

In [ ]:
#executable path for window users
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
#Executable path for Mac Users
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
#url of page to be scraped from indeed
url = 'https://www.indeed.com/jobs?q=Software+Engineer&l=Irvine%2C+CA&radius=100'
browser.visit(url)

In [ ]:
#BeautifulSoup object Parser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
#result to return the job title
titles = soup.find_all("h2", class_ = "title")
print(titles)
#title[1].get_text()

In [ ]:
#result to be Returned as an iterable list
for title in titles:
    job_title = title.get_text()
    print(job_title)
    

In [ ]:

try:
    dismiss = browser.find_by_xpath('/html/body/div[2]/div/section/div/div[2]')
    more_jobs = browser.find_by_xpath('/html/body/table[2]/tbody/tr/td/table/tbody/tr/td[1]/nav/div/ul/li[6]/a/span')
    no_thanks = browser.find_by_xpath('/html/body/div[3]/div[2]/div[4]/div[3]/div[2]/a')
    dismiss.first.click()
    more_jobs.first.click()
    no_thanks.first.click()
except:
    print("not found")

In [ ]:
#Empty list to append the the result and index 0 is an int 
job_titles = [0]
 #loop through the result
for x in range(1, 2):
    titles = soup.find_all("h2", class_ = "title")
    for title in titles:
        job_titles.append(title.text)
        print('page:', x, '-------------')
        print(title.text)
        browser.find_by_xpath('/html/body/table[2]/tbody/tr/td/table/tbody/tr/td[1]/nav/div/ul/li[6]/a/span/span')


In [ ]:
#Use replace to print out a clean job title
clean_job_titles = []

#loop through the result
for job in job_titles[1:]:
    clean_job_title = job.replace('\n', '')
    clean_job_titles.append(clean_job_title)

In [ ]:
#print clean job titles to check
print(clean_job_titles)

In [ ]:
#result to return the job locations
locations = soup.find_all("span", class_ = "location accessible-contrast-color-location")
locations

In [ ]:
#Empty list to append the result
job_locations = []

#loop through the returned result
for x in range(1, 2):
    locations = soup.find_all("span", class_ = "location accessible-contrast-color-location")
    for location in locations:
        job_locations.append(location.text)
        print('page:', x, '-------------')
        print(location.text)
        browser.find_by_xpath('/html/body/table[2]/tbody/tr/td/table/tbody/tr/td[1]/div[7]/div[1]/span[1]')
        

In [ ]:
#print job locations
print(job_locations)

In [ ]:
#zip both results into a list
new_list = list(zip(clean_job_titles, job_locations))

In [ ]:
#print new_list that has both the job_titiles and locations
print(new_list)

In [ ]:
#append list into a python dictionary
updated_list = dict(new_list)
updated_list

In [ ]:
#Initialize pymongo to work with MongoDBs
#CONNECT MONGO DB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
#convert newlist to pandas dataframe
updated_list_df = pd.DataFrame(new_list)
updated_list_df

In [ ]:
#append colums keys
New_job_location_df = pd.DataFrame(new_list, columns=['Job_Title', 'Location'])
New_job_location_df

In [ ]:
#Define database and collection
#Database = Indeed_db
#Collections = Software_jobs
client.Indeed_db.software_jobs.insert_many(New_job_location_df.to_dict('records'))

In [ ]:
#Done